## Libraries

In [11]:
import pandas as pd
import requests
from pathlib import Path
from functools import lru_cache
import spacy
from spacy.tokens import Doc

## Helper Functions

In [ ]:
@lru_cache(maxsize=None)
def download_csv(url, data_dir='./../data') -> pd.DataFrame:
    # Create cache directory if not exists
    Path(data_dir).mkdir(parents=True, exist_ok=True)
    
    # Generate a safe filename from URL
    filename = Path(data_dir) / url.split('/')[-1]
    
    # Check if file already exists locally
    if filename.exists():
        return pd.read_csv(filename)
    
    try:
        # Advanced request with timeout and proper headers
        headers = {
            'User-Agent': 'Mozilla/5.0 Academic Data Retrieval',
            'Accept': 'text/csv',
        }
        response = requests.get(
            url, 
            headers=headers, 
            timeout=300,  # 300 seconds timeout
            stream=True  # Memory efficient for large files
        )
        
        # Raise an exception for bad status codes
        response.raise_for_status()
        
        # Save to local cache
        with open(filename, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        
        return pd.read_csv(filename)
    
    except requests.exceptions.RequestException as e:
        print(f"Network error: {e}.")
        raise

In [16]:
def get_text_from_valid_tokens(doc: Doc) -> str:
    valid_tokens = []

    for token in doc:
        is_valid = token.is_alpha and not token.is_stop
        if is_valid:
            valid_tokens.append(token.text)

    if len(valid_tokens) > 2:
        return " ".join(valid_tokens)
    return ""

## Loading data

In [9]:
df_train = download_csv("https://cdn3.gnarususercontent.com.br/1638-word-embedding/treino.csv")
df_test = download_csv("https://cdn3.gnarususercontent.com.br/1638-word-embedding/teste.csv")

In [10]:
df_train.sample(5)

,title,text,date,category,subcategory,link
73487,NBA fala em 'injustiça' e estuda mudar sistema...,"O comissário da NBA, Adam Silver, disse nesta ...",2015-06-02,esporte,NaN,http://www1.folha.uol.com.br/esporte/2015/02/1...
35466,Gigantes da tecnologia usam clientes como escu...,Primeiro foi o diretor-geral do Google no país...,2016-01-03,mercado,NaN,http://www1.folha.uol.com.br/mercado/2016/03/1...
86848,"Lixo, tragédia e oportunidades",Podemos definir como trágica a situação atual ...,2016-06-04,colunas,raquelrolnik,http://www1.folha.uol.com.br/colunas/raquelrol...
7099,Flamengo critica árbitro que não marcou toque ...,"Nesta segunda-feira (7), o Flamengo publicou e...",2015-07-09,esporte,NaN,http://www1.folha.uol.com.br/esporte/2015/09/1...
24265,Dilma quer ampliar leilão de rodovias para até...,Em busca de um plano que gere impacto positivo...,2015-05-14,mercado,NaN,http://www1.folha.uol.com.br/mercado/2015/05/1...


## Processing Data

In [ ]:
train_texts = df_train["title"].str.lower()

In [17]:
nlp = spacy.load("pt_core_news_sm")

cleaned_texts = []
for doc in nlp.pipe(train_texts, batch_size=1000, n_process=-1):
    cleaned_texts.append(get_text_from_valid_tokens(doc))

In [18]:
df_train_cleaned = pd.DataFrame({"Title": cleaned_texts})
df_train_cleaned.sample(5)

,Title
62368,túmulo jesus restaurado apresentado jerusalém
24638,suprema corte eua avaliará legalidade casament...
76538,nevasca deixa retidos estação trem sul china
21524,greve argentinos dificuldade voltar país
79563,promotoria investigar clube lucra jogar campeo...
